In [1]:
import os
import shutil
from glob import glob
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_csv('./data/aug_train.csv')
df

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식
...,...,...,...,...,...,...
27345,aug_1_TRAIN_16367,./image/train\aug_1_TRAIN_16367.jpg,남원 목공예사업협동조합 판매장 수십년간 식상과 목기를 만들어온 장인들이 현대화시설의...,쇼핑,쇼핑,특산물판매점
27346,aug_2_TRAIN_16367,./image/train\aug_2_TRAIN_16367.jpg,남원 목공예사업협동조합 판매장 수십년간 식상과 목기를 만들어온 장인들이 현대화시설의...,쇼핑,쇼핑,특산물판매점
27347,aug_0_TRAIN_16556,./image/train\aug_0_TRAIN_16556.jpg,"지하1층,지상3층의 붉은 별돌 건물로 지하 1층에는 가든이 들어있고, 지상 1층에는...",쇼핑,쇼핑,특산물판매점
27348,aug_1_TRAIN_16556,./image/train\aug_1_TRAIN_16556.jpg,"지하1층,지상3층의 붉은 별돌 건물로 지하 1층에는 가든이 들어있고, 지상 1층에는...",쇼핑,쇼핑,특산물판매점


# label 전처리

In [3]:
label_df = pd.DataFrame(columns=['cat1','cat2','cat3','count'])
for n_cat1 in df['cat1'].unique() :
    cat2 = df[df['cat1'] == n_cat1]['cat2'].unique()
    for n_cat2 in cat2 :
        cat3  = df[df['cat1'] == n_cat1][df['cat2'] == n_cat2]['cat3'].unique()
        for n_cat3 in cat3 :
            cnt = len(df[df['cat1'] == n_cat1][df['cat2'] == n_cat2][df['cat3'] == n_cat3])
            n = pd.DataFrame([[n_cat1, n_cat2, n_cat3, cnt]], columns=['cat1', 'cat2', 'cat3', 'count'])
            label_df = pd.concat([label_df, n], ignore_index=True)

label_df

,cat1,cat2,cat3,count
0,자연,자연관광지,항구/포구,134
1,자연,자연관광지,섬,111
2,자연,자연관광지,자연휴양림,118
3,자연,자연관광지,해수욕장,207
4,자연,자연관광지,산,239
...,...,...,...,...
123,쇼핑,쇼핑,상설시장,270
124,쇼핑,쇼핑,"공예,공방",164
125,쇼핑,쇼핑,백화점,108
126,쇼핑,쇼핑,면세점,108


In [4]:
cat_encoder = {v : i for i, v in enumerate(label_df.loc[:, 'cat3'].values)}
cat_encoder

{'항구/포구': 0,
 '섬': 1,
 '자연휴양림': 2,
 '해수욕장': 3,
 '산': 4,
 '수목원': 5,
 '강': 6,
 '자연생태관광지': 7,
 '계곡': 8,
 '폭포': 9,
 '국립공원': 10,
 '해안절경': 11,
 '호수': 12,
 '동굴': 13,
 '도립공원': 14,
 '군립공원': 15,
 '약수터': 16,
 '등대': 17,
 '희귀동.식물': 18,
 '기암괴석': 19,
 '골프': 20,
 '야영장,오토캠핑장': 21,
 '스키(보드) 렌탈샵': 22,
 '자동차경주': 23,
 '자전거하이킹': 24,
 '썰매장': 25,
 '승마': 26,
 '트래킹': 27,
 '수련시설': 28,
 '카지노': 29,
 '번지점프': 30,
 '카트': 31,
 'MTB': 32,
 '스케이트': 33,
 '인라인(실내 인라인 포함)': 34,
 '사격장': 35,
 'ATV': 36,
 '빙벽등반': 37,
 '스키/스노보드': 38,
 '복합 레포츠': 39,
 '요트': 40,
 '래프팅': 41,
 '민물낚시': 42,
 '바다낚시': 43,
 '수영': 44,
 '카약/카누': 45,
 '윈드서핑/제트스키': 46,
 '스노쿨링/스킨스쿠버다이빙': 47,
 '스카이다이빙': 48,
 '헹글라이딩/패러글라이딩': 49,
 '수상레포츠': 50,
 '한식': 51,
 '일식': 52,
 '바/까페': 53,
 '채식전문점': 54,
 '중식': 55,
 '서양식': 56,
 '패밀리레스토랑': 57,
 '클럽': 58,
 '일반축제': 59,
 '문화관광축제': 60,
 '유적지/사적지': 61,
 '성': 62,
 '안보관광': 63,
 '사찰': 64,
 '종교성지': 65,
 '고택': 66,
 '고궁': 67,
 '민속마을': 68,
 '문': 69,
 '생가': 70,
 '전시관': 71,
 '미술관/화랑': 72,
 '박물관': 73,
 '도서관': 74,
 '공연장': 75,
 '기념관': 76,
 '

# FSL에 학습 규격에 맞춰서 이미지 이동

### 경로 만들기

In [5]:
root_path = './data/fsl_img_train'
os.makedirs(root_path, exist_ok=True)

for i, label_name in enumerate(label_df['cat3'].values) :
    fold_path = os.path.join(root_path, str(i))#label_name)
    os.makedirs(fold_path, exist_ok=True)

root_path = './data/fsl_img_valid'
os.makedirs(root_path, exist_ok=True)

for i, label_name in enumerate(label_df['cat3'].values) :
    fold_path = os.path.join(root_path, str(i))#label_name)
    os.makedirs(fold_path, exist_ok=True)

## 데이터 쪼개기

In [6]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=4, random_state=99, shuffle=True)
img_set = df['img_path']
label_set = df['cat3']

for train_ind, valid_ind in kfold.split(img_set, label_set) :
    train_df = df.iloc[train_ind]
    valid_df = df.iloc[valid_ind]
    break

### 이미지 이동

In [7]:
# df.iloc[0]
source_root_path = './data'
dest_root_path = './data/fsl_img_train'

for i in tqdm(range(len(train_df))):
    img_path = os.path.join(source_root_path, train_df.iloc[i]['img_path'])
    label = cat_encoder[train_df.iloc[i]['cat3']] #df.iloc[i]['cat3'] 
    img_name = train_df.iloc[i]['id']
    dest_path = os.path.join(dest_root_path, str(label), img_name+'.jpg')
    
    # copy
    shutil.copyfile(img_path, dest_path)

source_root_path = './data'
dest_root_path = './data/fsl_img_valid'

for i in tqdm(range(len(valid_df))):
    img_path = os.path.join(source_root_path, valid_df.iloc[i]['img_path'])
    label = cat_encoder[valid_df.iloc[i]['cat3']] #df.iloc[i]['cat3'] 
    img_name = valid_df.iloc[i]['id']
    dest_path = os.path.join(dest_root_path, str(label), img_name+'.jpg')
    
    # copy
    shutil.copyfile(img_path, dest_path)
    

100%|███████████████████████████████████████████████████████████████████████████████| 6838/6838 [00:09<00:00, 737.91it/s]


# FSL에 이용하는 json 파일 생성

In [8]:
import random
from glob import glob
from collections import defaultdict
import json

fsl_t = defaultdict(list)
fsl_v = defaultdict(list)

# vaild_pick = random.sample([i for i in range(128)], 26)

t_path = './data/fsl_img_train/*'
v_path = './data/fsl_img_valid/*'

t_label_list = glob(t_path)
t_label_list.sort(key=lambda x: int(x.split('\\')[-1]))

v_label_list = glob(v_path)
v_label_list.sort(key=lambda x: int(x.split('\\')[-1].split('.')[0]))



for idx, label in enumerate(t_label_list) :  
    fsl_t['class_roots'].append(label)
    fsl_t['class_names'].append(label.split('\\')[-1])


for idx, label in enumerate(v_label_list) :  
    fsl_v['class_roots'].append(label)
    fsl_v['class_names'].append(label.split('\\')[-1])

    
with open('./data/kfold_fsl_train.json', 'w') as f:
    json.dump(fsl_t, f, indent=2)
    
with open('./data/kfold_fsl_valid.json', 'w') as f:
    json.dump(fsl_v, f, indent=2)


# FSL configuration 설정

In [9]:
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm
import timm
import os

from easyfsl.methods.utils import evaluate
from easydict import EasyDict

args = EasyDict({'n_way' : 4, 
                 'n_shot' : 3, 
                 'n_query' : 6, 
                 'n_workers' : 6, 
                 'n_epochs' : 30,
                 'BATCH_SIZE' : 32, 
                 'RESIZE' : 300,
                 'TRAIN_PATH' : 'data/kfold_fsl_train',
                 'VALID_PATH' : 'data/kfold_fsl_valid',
                 'n_validation_tasks' :200,
                 'n_tasks_per_epoch' : 300,
                 'MODEL_NAME' : 'efficientnet_b3',
                 'MODEL_SAVE_PATH' : './ckpt/fsl/albumen_300_effib3',
                 'DEVICE' : "cuda"})

# DataLoader

In [10]:
from easyfsl.datasets import CUB, CUSTOM
from easyfsl.samplers import TaskSampler
from torch.utils.data import DataLoader


train_set = CUSTOM(split=args.TRAIN_PATH, image_size=args.RESIZE ,training=True)
val_set = CUSTOM(split=args.VALID_PATH, image_size=args.RESIZE, training=False)

train_sampler = TaskSampler(
    train_set, 
    n_way=args.n_way, 
    n_shot=args.n_shot, 
    n_query=args.n_query, 
    n_tasks=args.n_tasks_per_epoch
)
val_sampler = TaskSampler(
    val_set, 
    n_way=args.n_way, 
    n_shot=args.n_shot, 
    n_query=args.n_query, 
    n_tasks=args.n_validation_tasks
)

train_loader = DataLoader(
    train_set,
    batch_sampler=train_sampler,
    num_workers=args.n_workers,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)
val_loader = DataLoader(
    val_set,
    batch_sampler=val_sampler,
    num_workers=args.n_workers,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

# Model

In [11]:
class CNN(nn.Module) :
    def __init__(self, model_name='efficientnetv2_rw_m', num_classes=0)  :
        super(CNN, self).__init__()
        self.model = timm.create_model(model_name, pretrained=True, num_classes=num_classes, global_pool='')
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten(1)
        
    def forward(self, x) :
        x = self.model.forward_features(x)
        x = self.pool(x)
        x = self.flatten(x)
        return x

In [12]:
from easyfsl.methods import PrototypicalNetworks, FewShotClassifier, TIM, RelationNetworks

convolutional_network = CNN(model_name=args.MODEL_NAME).to(args.DEVICE)
few_shot_classifier = PrototypicalNetworks(convolutional_network).to(args.DEVICE)


# Episodic Train

In [13]:
from torch.optim import SGD, Optimizer, AdamW, Adam
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter


criterion = nn.CrossEntropyLoss()

scheduler_milestones = [3, 6]
scheduler_gamma = 0.1
learning_rate = 1e-3


train_optimizer = Adam(
    few_shot_classifier.parameters(), lr=learning_rate, weight_decay=5e-4)#,  momentum=0.9)


train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)


In [14]:
def training_epoch(
    model: FewShotClassifier, data_loader: DataLoader, optimizer: Optimizer
):
    all_loss = []
    model.train()
    with tqdm(
        enumerate(data_loader), total=len(data_loader), desc="Training"
    ) as tqdm_train:
        for episode_index, (
            support_images,
            support_labels,
            query_images,
            query_labels,
            _,
        ) in tqdm_train:
            optimizer.zero_grad()
            model.process_support_set(
                support_images.to(args.DEVICE), support_labels.to(args.DEVICE)
            )
            classification_scores = model(query_images.to(args.DEVICE))

            loss = criterion(classification_scores, query_labels.to(args.DEVICE))
#             loss.requires_grad_(True)
            loss.backward()
            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [15]:
from easyfsl.methods.utils import evaluate

start_e = 0
best_e = None
resum = False
model_save_path = './ckpt/fsl/epcisodic_albumen_300_effib3'
os.makedirs(model_save_path, exist_ok=True)

tb_logs_dir = Path(".")
tb_writer = SummaryWriter(log_dir='./tensorboard/epcisodic_albumen_224_effib3')


if resum == True : 
    fsl_checkpoint = torch.load(os.paht.join(model_save_path,'2E_model.pt'))
    few_shot_classifier.load_state_dict(fsl_checkpoint["model_state_dict"])
    train_optimizer.load_state_dict(fsl_checkpoint['optimizer_state_dict'])
    start_e = fsl_checkpoint["epoch"]
    
best_validation_accuracy = 0.0
for epoch in range(start_e, args.n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch(few_shot_classifier, train_loader, train_optimizer)
    validation_accuracy = evaluate(
        few_shot_classifier, val_loader, device=args.DEVICE, tqdm_prefix="Validation"
    )

    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        print("Ding ding ding! We found a new best model!")
        best_e = epoch
        torch.save({
                "epoch" : epoch,
                "model_state_dict" : few_shot_classifier.state_dict(),
                "optimizer_state_dict" : train_optimizer.state_dict()
            }, os.path.join(model_save_path, str(epoch)+'E_'+str(round(best_validation_accuracy, 4))+'_model.pt'))
    

    tb_writer.add_scalar("Train/loss", average_loss, epoch)
    tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()

Epoch 0


Validation: 100%|███████████████████████████████████████████████████████| 200/200 [00:26<00:00,  7.62it/s, accuracy=0.61]


Ding ding ding! We found a new best model!
Epoch 1


Validation: 100%|██████████████████████████████████████████████████████| 200/200 [00:26<00:00,  7.69it/s, accuracy=0.613]


Ding ding ding! We found a new best model!
Epoch 2


...

Validation: 100%|██████████████████████████████████████████████████████| 200/200 [00:26<00:00,  7.63it/s, accuracy=0.787]


Epoch 28


Validation: 100%|██████████████████████████████████████████████████████| 200/200 [00:25<00:00,  7.72it/s, accuracy=0.795]


Epoch 29


Validation: 100%|██████████████████████████████████████████████████████| 200/200 [00:25<00:00,  7.70it/s, accuracy=0.787]


# Classical Train

# Configuration

In [6]:
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm
import timm
import os

from easyfsl.methods.utils import evaluate
from easydict import EasyDict

args = EasyDict({'n_way' : 4, 
                 'n_shot' : 3, 
                 'n_query' : 6, 
                 'n_workers' : 6, 
                 'n_epochs' : 30,
                 'BATCH_SIZE' : 32, 
                 'RESIZE' : 300,
                 'TRAIN_PATH' : 'data/full_fsl_train',
                 'VALID_PATH' : 'data/CUB_fsl_valid',
                 'n_validation_tasks' :200,
                 'MODEL_NAME' : 'efficientnet_b3',
                 'MODEL_SAVE_PATH' : './ckpt/fsl/albumen_300_effib3',
                 'DEVICE' : "cuda"})

# DataLoader

In [7]:
from easyfsl.datasets import CUSTOM
from easyfsl.samplers import TaskSampler
from torch.utils.data import DataLoader


train_set = CUSTOM(split=args.TRAIN_PATH, image_size=args.RESIZE, training=True)
train_loader = DataLoader(
    train_set,
    batch_size=args.BATCH_SIZE,
    num_workers=args.n_workers,
    pin_memory=True,
    shuffle=True,
)


val_set = CUSTOM(split=args.VALID_PATH, image_size=args.RESIZE, training=True)

val_sampler = TaskSampler(
    val_set, 
    n_way=args.n_way, 
    n_shot=args.n_shot, 
    n_query=args.n_query, 
    n_tasks=args.n_validation_tasks
)
val_loader = DataLoader(
    val_set,
    batch_sampler=val_sampler,
    num_workers=args.n_workers,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

# Model

In [8]:
from easyfsl.methods import PrototypicalNetworks, FewShotClassifier, TIM, RelationNetworks

class CNN(nn.Module) :
    def __init__(self, model_name='efficientnet_b3', num_classes=0)  :
        super(CNN, self).__init__()
        self.model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten(1)
        
        self.use_fc = True
        
    def forward(self, x) :

        if self.use_fc:
            x = self.model(x)

        else :
            x = self.model.forward_features(x)
            x = self.pool(x)
            x = self.flatten(x)

        return x
    
    def set_use_fc(self, x) :
        self.use_fc = x
        
model = CNN(model_name=args.MODEL_NAME, 
            num_classes=len(set(train_set.get_labels()))).to(args.DEVICE)
print('num_classes = ',len(set(train_set.get_labels())))
few_shot_classifier = PrototypicalNetworks(model).to(args.DEVICE)

num_classes =  128


# Train

In [10]:
from torch.optim import SGD, Optimizer, AdamW, Adam
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter


criterion = nn.CrossEntropyLoss()

scheduler_milestones = [3, 6]
scheduler_gamma = 0.1
learning_rate = 1e-3
tb_logs_dir = './tensorboard/FSL_Classical_album_effib3'

import os
os.makedirs(args.MODEL_SAVE_PATH, exist_ok=True)

train_optimizer = Adam(
    few_shot_classifier.parameters(), lr=learning_rate, weight_decay=5e-4
)


train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

tb_writer = SummaryWriter(log_dir=tb_logs_dir)


In [11]:
def training_epoch(model_: nn.Module, data_loader: DataLoader, optimizer: Optimizer):
    all_loss = []
    model_.train()
    with tqdm(data_loader, total=len(data_loader), desc="Training") as tqdm_train:
        for images, labels in tqdm_train:
            optimizer.zero_grad()

            loss = criterion(model_(images.to(args.DEVICE)), labels.to(args.DEVICE))
            loss.backward()
            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [13]:
from easyfsl.methods.utils import evaluate


# best_state = model.state_dict()
best_validation_accuracy = 0.0
validation_frequency = 10

for epoch in range(args.n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch(model, train_loader, train_optimizer)

#     if epoch % validation_frequency == validation_frequency - 1:

        # We use this very convenient method from EasyFSL's ResNet to specify
        # that the model shouldn't use its last fully connected layer during validation.
    model.set_use_fc(False)
    validation_accuracy = evaluate(
        few_shot_classifier, val_loader, device=args.DEVICE, tqdm_prefix="Validation"
    )
    model.set_use_fc(True)

    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        best_state = model.state_dict()
        print("Ding ding ding! We found a new best model!")
        best_e = epoch
        torch.save({
                "epoch" : epoch,
                "model_state_dict" : best_state,
                "optimizer_state_dict" : train_optimizer.state_dict()
            }, os.path.join(args.MODEL_SAVE_PATH, str(epoch)+'E_'+str(round(best_validation_accuracy, 4))+'_model.pt'))
        
    tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    tb_writer.add_scalar("Train/loss", average_loss, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()

Epoch 0


Validation: 100%|███████████████████████████████████████████████████████| 200/200 [00:25<00:00,  7.96it/s, accuracy=0.54]


Ding ding ding! We found a new best model!
Epoch 1


Validation: 100%|██████████████████████████████████████████████████████| 200/200 [00:25<00:00,  7.84it/s, accuracy=0.518]


Epoch 2


...

Epoch 3


Validation: 100%|██████████████████████████████████████████████████████| 200/200 [00:25<00:00,  7.81it/s, accuracy=0.512]


Epoch 4


Training:  33%|████████████████████▎                                        | 181/543 [01:30<03:00,  2.01it/s, loss=1.32]


KeyboardInterrupt: 

In [3]:
a = [9,1,4,7,16,7]
a.sort()
print(a)

[1, 4, 7, 7, 9, 16]
